# Introducción al Procesado del Lenguaje Natural

### Aprendizaje Automático Intermedio e Ingeniería de Características

#### Febrero 2021

**Vanessa Gómez Verdejo, Emilio Parrado Hernández,  Pablo Martínez Olmos**

Departamento de Teoría de la Señal y Comunicaciones

**Universidad Carlos III de Madrid**

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [1]:
%matplotlib inline  
# Figures plotted inside the notebook
%config InlineBackend.figure_format = 'svg'  
# High quality figures
import matplotlib.pyplot as plt
import numpy as np

# Introducción

Hasta ahora hemos estado trabajando con datos de tipo  numérico o categórico. En esta sesión vamos a ver cómo trabajar con nuestros datos cuando éstos son cadenas de texto. A diferencia de los datos categóricos, en los que tenemos cadenas de texto asociadas a diferentes categorías y que podemos codificar fácilmente (por ejemplo, con un one-hot-encoding), cuando hablamos aquí de información textual nos referimos a frases, documentos y/o corpus de datos con estructura mucho más compleja. Idealmente, a partir de estos datos textuales tenemos que extraer la información necesaria (a poder ser incluyendo contenido semántico) y vectorizarla adecuadamente para poder utilizar o usar modelos de aprendizaje a partir de ella.

En general, gran parte de la forma en que nos comunicamos hoy en día es a través de texto escrito, ya sea en servicios de mensajería, medios sociales y/o correo electrónico. Así, por ejemplo, en servicios/aplicaciones como TripAdvisor, Booking, Amazon, etc., los usuarios escriben reseñas de restaurantes/negocios, hoteles, productos para compartir sus opiniones sobre su experiencia. Estas reseñas, todas escritas en formato de texto, contienen una gran cantidad de información que sería útil responder preguntas relevantes para el negocio usando métodos de aprendizaje automático, por ejemplo, para predecir el mejor restaurante en una determinada zona. 

Este tipo de tarea (preprocesado) se denomina **procesamiento del lenguaje natural** (Natural Language Processing, NLP).
El NLP es un subcampo de la lingüística, la informática y la inteligencia artificial que se ocupa de las interacciones entre los ordenadores (o procesadores) y el lenguaje humano; en particular engloba un conjunto de técnicas para permitir que los ordenadores procesen y analicen grandes cantidades de texto.



# Pipeline para el procesado de texto 

Como sabemos, los algoritmos de ML procesan números, no palabras, por lo que necesitamos transformar el texto en números significativos que contengan la información relevante de los documentos. Este proceso de convesión de texto a números es lo que llamaremos **vectorización**. 

No obstante, para tener una representación útil, se requieren normalmente algunos pasos de **preprocesamiento** previo que limpien y homogenizen los documentos: tokenización, eliminación de *stop-words*, lematización, etc.
La siguiente figura muestra los diferentes pasos que debemos seguir para procesar nuestros documentos hasta poder ser utilizados por nuestro modelo de aprendizaje:

<img src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/BBVA/NLP/PipelineNLP.png" width="80%"> 

A lo largo de este notebook, veremos las herramientas que tenemos disponibles en Python para llevar a cabo todos estos pasos. Concretamente, nos centraremos en el uso de dos librerias:
* [NLTK, Natural Language ToolKit](https://www.nltk.org/). Esta libreria es una excelente biblioteca de NLP escrita en Python por expertos tanto del mundo académico como de la industria. NLTK Permite crear aplicaciones con datos textuales rápidamente, ya que proporciona un conjunto de clases básicas para trabajar con corpus de datos, incluyendo colecciones de textos (corpus), listas de palabras clave, clases para representar y operar con datos de tipo texto (documentos, frases, palabras, ...) y funciones para realizar tareas comunes de NLP (conversión a token, conteo de palabras, ...). Por lo que va a ser de gran ayuda para el preprocesado de los documentos.
* [Gensim](https://pypi.org/project/gensim/) es otra librería de Python para el modelado por temáticas (*topic modeling*), la indexación de documentos y tareas de recuperación de la información para documentos. Está diseñada para operar con grandes cantidades de información (con implementaciones eficientes y paralelizables/distribuidas) y nos va a ser de gran ayuda para la vectorización de nuestros corpus de datos una vez preprocesados.

Empecemos este notebook cargando la librería NLTK y algunas de sus funcionalidades. A continuación, elegiremos un corpus de datos con el que empezar a analizar las funcionalidades básicas que aportan NLTK y Gensim y sobre el que veremos, uno por uno, en qué consisten los diferentes pasos de nuestro pipeline y cómo podemos implementarlos.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e056118\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e056118\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\e056118\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

# 1. Cargando nuestro corpus de datos

NLTK incluye diferentes corpus de datos  con los que podemos probar nuestras herramientas. Podemos encontrar información de todos ellos en [NLTK corpus](https://www.nltk.org/book/ch02.html).



## El objeto CorpusReader
Para empezar a trabajar vamos a usar el corpus **inaugural**, uno de los corpus de datos incluidos en NLTK y que consiste en 58 documentos de texto con los discursos presidenciales de los presidentes de EEUU.

La siguiente celda de código nos muestra cómo cargar el corpus...

In [ ]:
from nltk.corpus import inaugural
nltk.download('inaugural')
inaugural

Al cargar el corpus, se genera un objeto de tipo `CorpusReader`, llamado `inaugural`, con el contenido del mismo. Dado que un corpus es una colección de documentos/textos, podemos ver que documentos componen este corpus usando la función `.fileids()`.

In [ ]:
print(inaugural.fileids())

Una característica de este corpus es que los documentos que lo forman tienen información sobre el año de cada documento. Podemos crear una lista de los años de cada discurso usando [list comprenhension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python):


In [ ]:
years = [fileid[:4] for fileid in inaugural.fileids()]
print(years)

También podemos usar la función `.words()` de `inaugural` para acceder a las palabras que componen los documentos del corpus.

In [ ]:
print(inaugural.words())
print(len(inaugural.words()))

Si queremos podemos acceder a un **documento** concreto del corpus y extraer su contenido en crudo con la función `.raw()`. 

In [ ]:
trump_text = inaugural.raw('2017-Trump.txt')
trump_text

La varaible `trump_text` que obtenemos es un único string (con todas las funciones de los tipos string) que contiene todas las palabras del documento que hemos especificado. Por ejemplo, podemos ver los primeros 20 caracteres de este documento como:

In [ ]:
print(type(trump_text))

print(trump_text[:20])

print('\n The total number of characters in the document is %d' %(len(trump_text)))

El CorpusReader también nos permite cargar documentos estructurados por **frases**. Para ello tenemos que usar la función `.sents()`.

In [ ]:
trump_sents = inaugural.sents('2017-Trump.txt')
print('\n The total number of sentences in the document is %d' %(len(trump_sents)))
print(trump_sents[:5])

O, directamente, cargarlo a nivel de **palabras** (o **tokens**) usando el método `.words()`. Nótese que cuando hablamos de palabra o token no sólo son palabras con significado, sino que pueden ser números o signos de puntuación.

In [ ]:
trump_words = inaugural.words('2017-Trump.txt')
print('\n The total number of words in the document is %d' %(len(trump_words)))
print(trump_words[:5])

# 2. Preprocesado del corpus

Antes de transformar los datos de entrada de texto en una representación vectorial, necesitamos estructurar y limpiar el texto, y conservar toda la información que permita capturar el contenido semántico del corpus.

Para ello, el procesado típico de NLP aplica los siguientes pasos:

1. Tokenización
2. Homogeneización
3. Limpieza

## 2.1. Tokenization

**Tokenización** es el proceso de dividir el texto dado en piezas más pequeñas llamadas tokens. Las palabras, los números, los signos de puntuación y otros pueden ser considerados como tokens.

Ya hemos visto que el objeto `CorpusReader` incluye funciones para dividir el corpus en frases o palabras. Pero NLTK incluye también funciones genéricas para hacer estas operaciones sobre cualquier cadena de texto. En concreto, tiene dos funciones:
- `sent_tokenize`: es un tokenizador de frases. Este tokenizador divide un texto en una lista de oraciones. Para decidir dónde empieza o acaba una frase NLTK tiene un modelo pre-entrenado para el idioma específico en el que estemos trabajando. Este modelo lo hemos cargado al principio con `nltk.download('punkt')`.
- `word_tokenize`/`wordpunct_tokenize`:  Divide un texto en palabras u otros caracteres individuales cómo pueden ser signos de puntuación.

Veamos como funcionan estas funciones con el siguiente texto:

In [ ]:
texto = 'I\'m a dog and it\'s great! You\'re cool and Sandy\'s book is big. Don\'t tell her, you\'ll regret it! "Hey", she\'ll say!'

In [ ]:
sent=nltk.sent_tokenize(texto)
print(sent)

In [ ]:
sent_tokens1=nltk.wordpunct_tokenize(texto)
print(sent_tokens1)

In [ ]:
sent_tokens2=nltk.word_tokenize(texto)
print(sent_tokens2)

Aunque puede parecer que las funciones `wordpunct_tokenize` y `word_tokenize` hacen lo mismo, con este ejemplo vemos que `wordpunct_tokenize` permite separar los signos de puntuación mientras que `word_tokenize` no.  Nótese la diferencia al dividir `I'm` entre ambas funciones.

También podemos combinar `sent_tokenize` y `word_tokenize` para tener frases y cada frase divida en tokens:

In [ ]:
for sent in nltk.sent_tokenize(texto):
    print(nltk.wordpunct_tokenize(sent))
    print("**************")

#### Ejercicio 1: Tokenización del texto



Seleccionemos ahora uno de los textos de nuestro corpus y veamos cómo aplicar estas funciones:

In [ ]:
# Get a text
trump_text = inaugural.raw('2017-Trump.txt')

Complete la siguiente celda de código para dividir el texto en frases e imprima las 5 primeras frases.

In [ ]:
#<SOL>
#</SOL>

Complete las siguientes celdas de código para dividir el texto en tokens (considerando y sin considerar la puntuación) e imprima los primeros 5 tokens

In [ ]:
#<SOL>
#</SOL>

In [ ]:
#<SOL>
#</SOL>

## 2.2. Homogeneización

Al observar los tokens del corpus podemos ver que hay muchos tokens con algunas letras en mayúsculas y otras en minúsculas, el mismo token unas veces aparece en singular y otras en plural, o el mismo verbo que aparece en diferentes tiempos verbales. Para analizar semánticamente el texto, nos interesa  **homogeneizar** las palabras que formalmente son diferentes pero tienen el mismo significado.

Para ello podemos usar las herramientas de lematización de NLTK. El proceso habitual de homogeneización consiste en los siguientes pasos:

1. Eliminación de las mayúsculas y caracteres no alfanuméricos: de este modo los caracteres alfabéticos en mayúsculas se transformarán en sus correspondientes caracteres en minúsculas y  se eliminarán los caracteres no alfanuméricos, por ejemplo, los signos de puntuación.

2. Stemming/Lematización: eliminar las terminaciones de las palabras para preservar su raíz de las palabras e ignorar la información gramatical (eliminamos marcas de plurales, género, conjugaciones verbales, ...).

Veamos como ir aplicando uno a uno cada uno de estos pasos sobre el texto anterior una vez tokenizado por palabras.

In [ ]:
# Get and tokenize the text
trump_text = inaugural.raw('2017-Trump.txt')
trump_tokens=nltk.wordpunct_tokenize(trump_text)
print(trump_tokens)


#### **Ejercicio 2**: eliminación de mayúsculas y caracteres no alfanuméricos

Convierte todos los tokens de `trump_tokens` a minúsculas (usando el método `.lower()`) y elimina los tokens no alfanuméricos (que puedes detectar con el método `.isalnum()`). Este procesado puedes codificarlo una sola línea de código usando list comprehension.

In [ ]:
#<SOL>
#</SOL>

**Stemming and Lemmatización**

En el lenguaje común, las palabras pueden tomar diferentes formas indicando género, cantidad, tiempo (en el caso de los verbos), formas concretas para nombres/adjetivos o adverbios, ... Para muchas aplicaciones, es útil normalizar estas formas en alguna palabra canónica que facilite su análisis. Hay dos maneras de realizar este proceso:

1. El proceso de **stemming** reduce las palabras a su base o raíz 

      running --> run

      flowers --> flower

  Para poder hacer esta transformación necesitamos librerías específicas que tienen almacenadas para el vocabulario de cada idioma las raices de dicho vocabulario y hacen esta conversión. En NLTK hay varios stemmers disponibles:
  * Lancaster (inglés, es más reciente y bastante agresivo)
  * Porter (inglés, es el stemmer original)
  * Snowball  (incluye muchos idiomas y es el más nuevo)

    
2. El objetivo de la **lematización**, al igual que el stemmer, es reducir las formas inflexionales a una forma base común. A diferencia del steming, la lematización no se limita a cortar las inflexiones. En su lugar, utiliza bases de conocimiento léxico para obtener las formas básicas correctas de las palabras. 

    women   --> woman

    foxes   --> fox
    
  La lematización en NLTK se basa en el léxico de [WordNet](https://wordnet.princeton.edu/). WordNet es un diccionario de inglés de orientación semántica, incluye el inglés WordNet con 155.287 palabras y 117.659 conjuntos de sinónimos. 

Veamos cómo funcionan el stemming y la lematización con un ejemplo:   

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer

text = list(nltk.word_tokenize("The women running in the fog passed bunnies working as computer scientists."))

# Load several stemmers 
snowball = SnowballStemmer('english')
lancaster = LancasterStemmer()
porter = PorterStemmer()

for stemmer in (snowball, lancaster, porter):
    stemmed_text = [stemmer.stem(t) for t in text]
    print(" ".join(stemmed_text))

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

# Try lemmatizer
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(t) for t in text]
print(" ".join(lemmas))

Compara cómo los diferentes procesos transforman palabras como `women`, `running` o `computer`.

Una de las ventajas de la lematización es que el resultado sigue siendo una palabra, lo que es más aconsejable para la presentación de los resultados del procesado de textos.

Sin embargo, sin utilizar información contextual, `lemmatize()` no elimina las diferencias gramaticales. Por esta razón, "running" o "passed" se conservan y no se sustituyen por el infinitivo "run" o "pass".

Como alternativa, podemos aplicar `.lemmatize(word, pos)`, donde `pos` es un código de cadena que especifica función gramatical de las palabras en su oración. Por ejemplo, se puede comprobar la diferencia entre `wnl.lemmatize('running')` y `wnl.lemmatize('running', pos='v')`.


In [ ]:
print(lemmatizer.lemmatize('running'))
print(lemmatizer.lemmatize('running', pos='v'))

Notese que ninguno de los dos da una solución perfecta... al final este proceso requiere de supervisión manual que nos permita afinar esta homogeneización lo mejor posible. En ocasiones lo que se hace es que se aplica un primer lematizado y sobre un stemming:

In [ ]:
stemmed_text = [snowball.stem(t) for t in text]
lemmas_text = [lemmatizer.lemmatize(t) for t in stemmed_text]
print(lemmas_text)

#### **Ejercicio 3**: Stemming/Lematización

Aplique el proceso de stemming y lematización sobre el texto del discurso inaugural de Trump resultantes del proceso de filtrado anterior (salida del Ejercicio 2).

In [ ]:
#<SOL>
#</SOL>

## 2.3. Limpieza

El último paso del preprocesado consiste en eliminar las palabras irrelevantes o **stop words** de los documentos. Las **stop words** son las palabras más comunes en un idioma como "el", "a", "sobre", "es", "todo". Estas palabras no tienen un significado importante y normalmente se eliminan de los textos. Para aplicar este proceso, se cargan librerías específicas que contienen este listado de palabras por cada idioma.

Además, este paso suele utilizarse para eliminar las marcas de puntaución (',', '.', '?', ....), para lo que también tenemos que cargar otro módulo con los signos de puntuación. 


In [ ]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')
print(stopwords_en[:100])

In [ ]:
import string
punctuation = string.punctuation
print(punctuation)


Veamos como aplicarlo con un ejemplo:

In [ ]:
text = nltk.word_tokenize('I\'m a dog and it\'s great! You\'re cool and Sandy\'s book is big. Don\'t tell her, you\'ll regret it! "Hey", she\'ll say!')
print(text)

In [ ]:
clean_text1 = [token for token in text if (token not in stopwords_en)] 
print(clean_text1)   

In [ ]:
clean_text2 = [token for token in text if (token not in punctuation)] 
print(clean_text2)   

#### **Ejercicio 4**: Eliminación de stop-words y puntuación

Aplique conjuntamente el proceso de eliminación de stop-words y puntuación sobre el texto del discurso inaugural de Trump resultantes del proceso de filtrado y lematización anterior (salida Ejercicio 3).

In [ ]:
print('Texto filtrado y lematizado:')
print(trump_tokens_stem)
#<SOL>
#</SOL>

Nota: En el primer paso ya eliminamos las marcas de puntuación con la función es `.isalnum()`; por lo que no sería necesario eliminar de nuevo la puntuación. 

## 2.4 Pipeline de preprocesado o normalización del texto

Por último, y para facilitar su uso posterior, vamos a juntar estos tres pasos en una única función que nos permita realizar la tokenización, homogeneización y limpieza con una única llamada a una función.

#### **Ejercicio 5**: Función para la normalización de textos

Complete el código de la siguiente función para poder hacer todos los pasos anteriores en una única función y luego pruebe a utilizarla sobre el texto del discurso inaugural de Trump

In [ ]:
## Load Modules
lemmatizer  = WordNetLemmatizer()
snowball = SnowballStemmer('english')
stopwords   = set(nltk.corpus.stopwords.words('english'))
punctuation = string.punctuation

def normalize(text):
    normalized_text = []
    for token in nltk.word_tokenize(text):
        #<SOL>
#</SOL>

In [ ]:
print('Número de documentos en el corpus preprocesado:')
print(len(corpus_prec))
print('**********')
print('Algunos de los elementos del primer documento preprocesado')
print(corpus_prec[0][:20])
print('**********')
print('Algunos de los elementos del segundo documento preprocesado')
print(corpus_prec[1][:20])
print('**********')
print('Algunos de los elementos del tercer documento preprocesado')
print(corpus_prec[3][:20])

# 3. Vectorización

Hasta este punto, hemos transformado la colección de textos en bruto en una lista de textos, en la que cada texto es una colección de las raíces de las palabras más relevantes para el análisis semántico. Ahora, necesitamos convertir estos datos (una lista de listas de tokens) en una representación numérica (una lista de vectores, o una matriz). 

Antes de pasar a hacer esta vectorización, documento a documento, vamos a hacer un **análisis frecuencial** del contenido del corpus preprocesado. Para ello vamos a obtener:
- Un recuento de palabras: número de veces que aparece cada palabra en el corpus.
- El vocabulario: conjunto de palabras únicas dentro del corpus.
- La diversidad léxica: la relación entre el número de palabras y el vocabulario.

Y para ello vamos a usar la dos clases muy útiles de NLTK que nos permiten hacer estos análisis de frecuencia:

- `FreqDist`
- `ConditionalFreqDist` 


## 3.1 Análisis de frecuencias del corpus

Para empezar a hacer este análisis, vamos a convertir nuestra lista de documentos, donde cada documento tiene una lista de tokens, en una única lista con todos los tokens del corpus. Una vez hecha esta conversión, utilizamos `FreqDist` y algunos de sus métodos para analizar frecuencialmente el contenido del corpus.

In [ ]:
tokens_corpus = [token for doc in corpus_prec for token in doc]
counts  = nltk.FreqDist(tokens_corpus)

Podemos comprobar que `counts` es un diccionario que contiene el número de veces que cada palabra aparece en el corpus

In [ ]:
counts # counts is a FreqDist object, a dictionary data type with additional methods

In [ ]:
counts['citizen']  

Simplemente operando sobre `counts` podemos calcular fácilmente el tamaño del vocabulario y la diversidad léxica:

In [ ]:
vocab   = len(counts.keys()) 
words   = sum(counts.values())
lexdiv  = float(words) / float(vocab)

print("El corpus tiene %i palabras únicas y un total de %i palabras con una diversidad léxica de %0.3f" % (vocab, words, lexdiv))

Con el método  `most_common(n)` obtenemos una lista de las palabras más comunes. Haciendo `counts.plot(n,cumulative=True)` podemos tener una idea de cuánto dominan las palabras más comunes en el corpus:

In [ ]:
counts.most_common(20)

In [ ]:
counts.plot(40, cumulative=False)

Otra forma de analizar la frecuencia de aparición de las palabras en el corpus es dibujando una nube llena de muchas palabras de diferentes tamaños, que representan la frecuencia o la importancia de cada palabra. Esto es lo que se conoce como **word cloud** o **nube de palabras**.

In [ ]:
from wordcloud import WordCloud

# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=50, max_words=40, background_color="white").generate(' '.join(tokens_corpus))

# Display the generated image:
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## 3.2 Bag of Words (BOW)

Si este análisis de frecuencia lo hicieramos a nivel de documento, en vez de a nivel de corpus, nos estaría indicando la ocurrencia de cada palabra por documento. De hecho, este conteo de palabras por documento es lo que se conoce como **bag-of-words** (bolsa de palabras) o, de manera abreviada como , **BoW**. En concreto, **bag-of-words** es una representación de texto que describe la ocurrencia de palabras dentro de un documento. 

Se llama "bolsa" de palabras, porque cualquier información sobre el orden o la estructura de las palabras en el documento es descartada. El modelo sólo se ocupa de si las palabras conocidas aparecen en el documento, no de dónde aparecen en el documento.

Para obtener la representación BoW podemos utilizar varias librerías. `Sklearn` incluye funcionalidades para el análisis de texto, y las propias funcionalidades de NLTK para el conteo de palabras o análisis frecuencial de palabras podrían valernos. Sin embargo, en esta sección vamos a ver cómo usar la librería de [`gensim`](https://pypi.org/project/gensim/) diseñada específicamente para el procesado de texto, permitiendo un procesado muy eficiente para grandes corpus e incluyendo funcionalides adicionales de gran utilidad. 


El proceso de generación del BoW tiene dos pasos:
1. Generación del vocabulario del corpus. Este vocabulario se genera mediante un **diccionario** que almacena de manera ordenada un vocabulario de palabras conocidas (a nivel de corpus).
2. **Vectorización de documentos** o generación del BoW. Se calcula una medida de la presencia de palabras conocidas (las que están en el vocabulario); para ello, realiza  un conteo del número de veces que cada palabra del diccionario aparece dentro de cada documento.

Veamos a continuación, como implementar estos pasos: 


### Generación del diccionario

In [ ]:
import gensim

# Create dictionary of tokens: the input is the preprocessed corpus 
D = gensim.corpora.Dictionary(corpus_prec)
n_tokens = len(D)

print('The dictionary contains', n_tokens, 'terms')
print('First terms in the dictionary:')
for n in range(10):
    print(str(n), ':', D[n])

Como vemos el diccionario no es más que una lista de palabras. Pero el orden de esta lista será muy importante para la vectorización de cada documento, ya que generaremos tuplas de (`id`, `conteo`) y los `id` serán las posiciones de estas palabras en el diccionario.

### Gestión del Vocabulario

A medida que el tamaño del vocabulario aumenta, también lo hace la representación vectorial de los documentos. En el ejemplo anterior, la longitud del vector de los documentos es igual al número de palabras conocidas.

Para un corpus muy grande, como miles de documentos, la longitud del vector que representará cada documento puede ser de miles o millones de posiciones. 

Además, cada documento puede contener muy pocas de las palabras conocidas en el vocabulario, lo que dificulta el procesado. 

Para evitar esto, es recomendable analizar el diccionario y eliminar tokens que no resulten relevantes: marcas de puntuación que todavía queden o términos que aparecen en muy pocos casos (por lo que no son términos poco informativos) o, incluso, tokens que figuran en todos los documentos del corpus (y no son discriminativos). 




El diccionario de Gensim nos permite filtrar estos terminos fácilmente:

* `.filter_tokens()` nos permite indicar con el parámetro `bad_ids` la lista de ids de las palabras a eliminar o con `good_ids` la lista de los ids de las palabras a dejar en el diccionario (eliminando el resto).

* `.filter_extremes()` permite eliminar palabras/tokens poco o muy frecuentes indicando con los parámetros:
  * `no_below`: se queda con el número de tokens que están contenidas en al menos `no_below` documentos.
  * `no_above`:  se queda con el porcentaje  (fracción del tamaño total del corpus, no un número absoluto) de tokens que no están en más de `no_above` documentos.
  * `keep_n`: directamente se queda con los `keep_n` tokens más frecuentes.
  * `keep_tokens`: listado de tokens que deben permanecer en el diccionario después de ser filtradas.

Veamos como utilizar estas dos funcionalidades...

In [ ]:
D.filter_tokens(bad_ids=[0])  # quitamos del diccionario el término "14th"
n_tokens = len(D)
print('The dictionary contains', n_tokens, 'terms')
print('First terms in the dictionary:')
for n in range(10):
    print(str(n), ':', D[n])

In [ ]:
no_below = 5 #Minimum number of documents to keep a term in the dictionary
no_above = .75 #Maximum proportion of documents in which a term can appear to be kept in the dictionary

D.filter_extremes(no_below=no_below,no_above=no_above, keep_n=1500)
n_tokens = len(D)

print('The dictionary contains', n_tokens, 'terms')

print('First terms in the dictionary:')
for n in range(10):
    print(str(n), ':', D[n])

Esto elimina todas los tokens en el diccionario que son:
1. Los `no_below=5` tokens menos frecuentes  
2. El `no_above=0.75` ($75\%$) de tokens más frecuentes.
3. Después de (1) y (2), guarda sólo los primeros `keep_n` tokens más frecuentes (o guarda todos si `keep_n=None`).



#### **Ejercicio 7**: Refinando el diccionario
A continuación, vamos a volver a generar el diccionario de nuestro corpus y analizar la aparición de palabras por documento. A partir de este análisis decida qué palabras eliminaría del diccionario y use las funciones anteriores para refinar el diccionario.

Para este análisis usamos algunos de los métodos de la clase diccionario de gensim como `dfs` (calcula en cuántos documentos del corpus aparece cada palabra del vocabulario). Puede ver la ayuda en este [link](https://radimrehurek.com/gensim/corpora/dictionary.html).

In [ ]:
# Recompute the dictionary
D = gensim.corpora.Dictionary(corpus_prec)
n_tokens = len(D)
print(n_tokens)

In [ ]:
# Analize the word frequency per document 
plt.hist(D.dfs.values(),58)
plt.show()
# we have 58 documents, some words are in all of them -> no_above= 0.8% 
# There are many words in a single document -> no_below = 5

In [ ]:
# Incluya aqui su código para limpiar el diccionario
#<SOL>
#</SOL>

In [ ]:
n_doc=50
print('Original document (after cleaning):')
print(corpus_prec[n_doc])
print('Sparse vector representation (first 10 components):')
print(corpus_bow[n_doc][:10])
print('Word counts for the document (first 10 components):')
list_word_counts = [(D[doc_bow[0]], doc_bow[1]) for doc_bow in corpus_bow[n_doc][:10]]
print(list_word_counts)


In [ ]:
(words, counts) = zip(*list_word_counts)
plt.figure(figsize=(10,5))
plt.stem(words,counts, use_line_collection = True)
plt.xlabel('Words')
plt.ylabel('Counts')
plt.show()

## 3.3 Representación TF-IDF 

Un problema del BoW es que la frecuencia de las palabras muy frecuentes empiezan a dominar en el documento respecto al resto; por ejemplo, verbos muy comunes o términos habituales en el contexto del corpus pero no tienen tanto "contenido informativo" para el modelo como pueden ser palabras más raras pero tal vez específicas del dominio. Al realizar la gestión del vocabulario hemos visto que podemos eliminar estas palabras tan frecuentes, pero este proceso es bastante manual y es preferible tener una vectorización robusta a esto.

Para ello, la representación TF-IDF (Term Frequency–Inverse Document Frequency) propone reajustar la frecuencia de las palabras en función de la frecuencia con que aparecen en todos los documentos, de modo que se penalicen las puntuaciones de palabras frecuentes si también son frecuentes en todos los documentos. Para ello, el TF-IDF implica el cálculo de dos valores:


**Frecuencia de término (TF)**

Por frecuencia de término $\text{TF}(w)$ nos referimos al número de veces que una palabra $w$ dada ocurre en un documento, dividido por el número total de palabras en dicho documento.
$$ \text{TF}(w,d) =\frac{\text{# veces que $w$ aparece en el documento $d$}}{\text{# total de palabras en el documento $d$}}$$

**Frecuencia de Documento Inversa (IDF)**

Es una medida de cuánta información proporciona la palabra $w$, es decir, si es común o rara en todos los documentos del corpus $D$. Se calcula de la siguiente manera:
$$ \text{IDF}(w,D) =\log \frac{\text{# palabras en el corpus}}{1+\text{# documentos donde la palabra $w$ aparece}}$$
 
A partir de estos valores el **TF-IDF** se calcula de la siguiente manera:

$$\text{TF-IDF}(w,d,D) = \text{TF}(w,d) * \text{IDF}(w,D)$$

Un peso alto en TF-IDF se consigue cuando la palabra tiene una frecuencia alta en el documento y, a la vez, una frecuencia baja en el corpus; por lo tanto, los pesos tienden a filtrar los términos que son comunes a muchos documentos. 

Obsérvese que, a diferencia de la codificación BoW, para el TF-IDF tenemos que parender la codificación conjuntamente con todo el corpus. No obstante, una vez que hemos calculado el BoW para todos los documentos, aprender el modelo TF-IDF es sencillo usando la función [TfidfModel](https://radimrehurek.com/gensim/models/tfidfmodel.html) de Gensim.

In [ ]:
from gensim.models import TfidfModel

# fit TFIDF model for all the corpus
model = TfidfModel(corpus_bow)  

# apply model to the first corpus document
vector = model[corpus_bow[0]]  # apply model to the first corpus document
print(vector)



#### **Ejercicio 8**: Obtenga el TFIDF de todos los documentos del corpus

Almacene la salida en una lista llamada `corpus_tfifd` donde cada elemento de la lista sea el TFIDF de un documento, para a continuación poder analizar la salida y representarla.

In [ ]:
#<SOL>
#</SOL>

Analicemos esta transformación en comparación con el BoW

In [ ]:
n_doc=50
print('Original document (after cleaning):')
print(corpus_prec[n_doc])
print('Sparse TFIDF vector representation (first 10 components):')
print(corpus_tfidf[n_doc][:10])
print('Word counts for the document (first 10 components):')
list_word_counts = [(D[doc_bow[0]], doc_bow[1]) for doc_bow in corpus_bow[n_doc][:10]]
print(list_word_counts)
print('TF-IDF for the document (first 10 components):')
list_tfidf = [(D[doc_tfidf[0]], doc_tfidf[1]) for doc_tfidf in corpus_tfidf[n_doc][:10]]
print(list_tfidf)

In [ ]:
#Plot BoW
(words, counts) = zip(*list_word_counts)
plt.figure(figsize=(10,5))
plt.stem(words,counts, use_line_collection = True)
plt.xlabel('Words')
plt.ylabel('Counts')
plt.show()

In [ ]:
#Plot Tf-IDF
(words, counts) = zip(*list_tfidf)
plt.figure(figsize=(10,5))
plt.stem(words,counts, use_line_collection = True)
plt.xlabel('Words')
plt.ylabel('Counts')
plt.show()

Compare ambas representaciones... ¿Qué palabras tenían más/menos peso en la representación BoW? ¿Y en la representación TF-IDF? 

## Limitaciones de BoW y TF-IDF

Las representaciones vectoriales que acabamos de ver son sencillas de comprender y aplicar y ofrecen mucha flexibilidad para manejo de información textual. De hecho, se han utilizado con gran éxito en problemas de predicción y clasificación de documentos.

Sin embargo, sufre de algunas deficiencias que tenemos que tener en cuenta:
* El vocabulario: El vocabulario requiere un diseño cuidadoso, más específicamente para gestionar su tamaño, lo que afecta a la dispersión de las representaciones de los documentos.
* La dispersión: Las representaciones dispersas son más difíciles de modelar tanto por razones computacionales como por razones de información, en las que el reto es que los modelos aprovechen la poca información común en un espacio de representación tan grande.
* Significado: Al descartar el orden de las palabras se ignora el contexto y, a su vez, el significado de las palabras del documento (semántica). El contexto y el significado pueden ofrecer mucho al modelo, que si se modela podría diferenciar entre las mismas palabras dispuestas de manera diferente ("esto es interesante" vs "es esto interesante?"), sinónimos ("bicicleta vieja" vs "bicicleta usada"), y mucho más.


# 4. Utilizando la representación vectorial en modelos de aprendizaje

Una vez tenemos una representación vetorial para cada documento del corpus podemos usar los modelos de aprendizaje que conocemos para resolver diferentes tareas. Por ejemplo, si este corpus estuviese distribuido por categorías, podríamos aprender un modelo para clasificar automáticamente los textos en las diferentes categorías. Como el corpus con el que estamos trabajando no está etiquetado, vamos a usar la representación vectorial que hemos obtenido para hacer un agrupamiento de los documentos usando un K-means y luego analizar qué información nos da este agrupamiento.

## 4.1 Codificación de los datos

Primero de todo, para poder usar las librerías de sklearn, tenemos que convertir nuestra representación vectorial en numpy arrays. Para ello, gensim nos incluye dos funciones: [corpus2dense](https://tedboy.github.io/nlps/generated/generated/gensim.matutils.corpus2dense.html), [corpus2csc](https://tedboy.github.io/nlps/generated/generated/gensim.matutils.corpus2csc.html)

In [ ]:
from gensim.matutils import corpus2dense, corpus2csc

n_tokens = len(D)
num_docs = len(corpus_bow)
# Convert BoW representacion
corpus_bow_dense = corpus2dense(corpus_bow, num_terms=n_tokens, num_docs=num_docs).T
corpus_bow_sparse = corpus2csc(corpus_bow, num_terms=n_tokens, num_docs=num_docs).T
# Convert TFIDF representacion
corpus_tfidf_dense = corpus2dense(corpus_tfidf, num_terms=n_tokens, num_docs=num_docs).T
corpus_tfidf_sparse = corpus2csc(corpus_tfidf, num_terms=n_tokens, num_docs=num_docs).T

Veamos qué han hecho estas transformaciones, por ejemplo, para la representación BoW: 

In [ ]:
print(corpus_bow[50])

In [ ]:
print(corpus_bow_dense[50,:])

In [ ]:
print(corpus_bow_sparse[50,:])

En general, cuando tengamos grandes corpus de datos, nos interesará manejar la representación dispersa o sparse de los datos para ahorrar en coste computacional. Si en nuestro procesado de ML usamos sklearn podremos trabajar con este formato sparse ya que la mayoría de clasificadores/regresores puede trabajar con ambos datos densos y dispersos.

Pasemos ahora a aplicar un agrupamiento o clustering de nuestros documentos usando ambas representaciones vectoriales (BoW y TFIDF). En ambos casos usaremos la representación sparse de los datos.

## 4.2 Calculo de distancias entre representaciones BoW y TF-IDF

Una vez que tenemos la representación vectorial de nuestros documentos (cada una de las filas de la matriz BoW o tfidf), muchos de nuestros modelos de ML necesitarán calcular similitudes entre ellos, como puede ser un algoritmo K-NN para regresión o un K-means para agrupamiento. Para ello, cuando trabajamos  con características BoW o TFIDF, donde la magnitud de los vectores no importa, suele utilizarse como métrica la **similitud del coseno**. 

Podríamos suponer que cuando una palabra (por ejemplo, ciencia) aparece con más frecuencia en el documento 1 que en el 2, ese documento 1 está más relacionado con el tema de la ciencia. Sin embargo, también podría darse el caso de que estemos trabajando con documentos de longitudes desiguales (artículos de Wikipedia, por ejemplo). En ese caso, es probable que la ciencia aparezca más en el documento 1 sólo porque es mucho más largo que el documento 2. La similitud del coseno corrige esto.

Por esta razón, cuando se trabaja con documentos codificados con BoW o TF-IDF, se tiende a utilizar la similitud del coseno. Si $\mathbf{v}_1$ y $\mathbf{v}_2$ son dos vectores TF-IDF, la similitud del coseno se calcula como sigue:


$$\text{cos}(\mathbf{v}_1,\mathbf{v}_2) = \frac{\mathbf{v}_1^T~\mathbf{v}_2}{||\mathbf{v}_1||_2 ||\mathbf{v}_2||}$$

O, más bien se usa la distancia coseno definda como $1-\text{cos}(\mathbf{v}_1,\mathbf{v}_2) $, es decir,
$$d_\text{cos}(\mathbf{v}_1,\mathbf{v}_2) = 1- \frac{\mathbf{v}_1^T~\mathbf{v}_2}{||\mathbf{v}_1||_2 ||\mathbf{v}_2||}$$



<img src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/BBVA/NLP/NLP1.png" width="100%"> 

Y como podemos ver en la siguiente figura, sus resultados no tienen nada que ver con la distancia euclídea (no dependen de la longitud de los vectores)

<img src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/BBVA/NLP/NLP2.png" width="60%"> 


Otro aspecto que tenemos que tener en cuenta es que la distancia coseno no es una distancia como tal (no cumple la desigualdad triangular) e implementaciones por defecto de nuestros modelos de aprendizaje como el K-NN o K-means de sklearn usan la distancia euclídea por defecto y no permiten incluir la distancia coseno. 

Para solventar este problema tenemos dos opciones:
* Usar otras implementaciones como K-means de NLTK que nos permite usar la distancia coseno. Aunque el problema de esta implementación es que no nos deja trabajar con matrices sparse. 
* Reescalar nuestros datos para poder utilizar implementaciones basadas en la distancia euclídea... (luego explicaremos en qué consiste esta aproximación).



## 4.3 ¿Normalización de los datos?

Hasta ahora siempre hemos visto en nuestro pipeline de ML que debemos normalizar los datos por columnas (por características) antes de pasarlos al modelo de aprendizaje.

Pero trabajando con representaciones vectoriales de texto como BoW o TF-IDF, ¿necesitamos normalizar? Aunque no hay una respuesta cerrada, la idea más aceptada es que las representaciones BoW o TF-IDF u otras transformaciones NLP que veremos más adelante deberían dejarse tal y como están para obtener mejores resultados.

Esto se debe a que la normalización de las variables BoW no se considera algo natural, perdemos la importancia de cada palabra dentro del documento. Mientras que la codificación TF-IDF ya se considera que está normalizada; nótese que:
* El cálculo del TF normaliza cada documento a longitud 1 (normalización por filas), eliminado la dependencia con documentos más largos o más cortos. 
*  En segundo lugar, el IDF es una normalización entre documentos que da menos peso a los términos comunes y más a los raros (normalizacióón por columnas), normalizando (ponderando) cada palabra con la frecuencia inversa del corpus.

Por lo que el TF-IDF está pensado para ser utilizado en su forma directa en un algoritmo. 

No obstante, tal y como veremos, algoritmos como el K-means que suelen ser muy sensibles al escalado de características la ponderación IDF ayuda a mejorar el resultado del agrupamiento y, por tanto, suelen obtenerse resultados mejores con representaciones TF-IDF que con BOW.

## 4.4 K-means con NLTK

Empecemos haciendo agrupamiento de datos usando la librería de NLTK que nos permite usar la distancia coseno. 

Aunque, tal y como hemos comentado, el problema de esta implementación es que no nos deja trabajar con matrices sparse; así que tenemos que usar el método `.toarray()` para convertir nuestros vectores sparse en numpy arrays (o usar la conversión a matrices densas de gensim).

In [ ]:
# Transform data to dense
corpus_bow_array = corpus_bow_sparse.toarray()
corpus_tfidf_array = corpus_tfidf_sparse.toarray()

Ahora entrenaremos el K-means con 4 centroides, tanto para la representación BoW como TF-IDF y analizaremos los resultados. 

**Clustering de documentos con BoW**

In [ ]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer

K=4
kclusterer = KMeansClusterer(4, distance=nltk.cluster.util.cosine_distance, repeats=10)

y_kmeans = kclusterer.cluster(corpus_bow_array, assign_clusters=True)
centers = kclusterer.means()

¿Qué documentos están en cada grupo? ¿Qué información (textual) hay en cada grupo?

In [ ]:
# Palabras más relevantes por centroide
n_center = 0
fileids = inaugural.fileids()
y_kmeans = np.array(y_kmeans)
for i, center_i in enumerate(centers):
  print('Centroide %d' %(i))
  # Find documents in this centroid
  pos_y = np.where(y_kmeans==i)[0]
  print('Elementos grupo %d' %(i))
  id_grupo = [fileids[pos] for pos in pos_y]
  print(id_grupo)

  # Find the ten most relevant words for this centroid
  pos_sort = np.argsort(centers[i])[::-1][:10]
  words_center = [D[p] for p in pos_sort]
  print(words_center)
  print('********')


**Clustering de documentos con TF-IDF**

In [ ]:
K=4
kclusterer = KMeansClusterer(4, distance=nltk.cluster.util.cosine_distance, repeats=10)

y_kmeans = kclusterer.cluster(corpus_tfidf_array, assign_clusters=True)
centers = kclusterer.means()

In [ ]:
# Palabras más relevantes por centroide
n_center = 0
fileids = inaugural.fileids()
y_kmeans = np.array(y_kmeans)
for i, center_i in enumerate(centers):
  print('Centroide %d' %(i))
  # Find documents in this centroid
  pos_y = np.where(y_kmeans==i)[0]
  print('Elementos grupo %d' %(i))
  id_grupo = [fileids[pos] for pos in pos_y]
  print(id_grupo)

  # Find the ten most relevant words for this centroid
  pos_sort = np.argsort(centers[i])[::-1][:10]
  words_center = [D[p] for p in pos_sort]
  print(words_center)
  print('********')


Analice y discuta los resultados de ambos clusters para cada tipo de vectorización.

## 4.5 K-means con sklearn

Como hemos dicho anteriormente, no podemos usar las librerías de K-means o K-NN de sklearn con represntaciones BOW o TF-IDF porque solo permiten usar la distancia euclídea. 

Sin embargo, cuando nuestros vectores $x$ e $y$ están normalizados ($||x||^2 = x^Tx = 1$), sus distancias euclídeas $||x-y||^2$  y coseno $d(x,y)$ están relacionas por la siguiente igualdad:

$$||x-y||^2 = x^Tx + y^Ty - 2 x^Ty = 2  (1 - x^Ty) = 2  d(x,y)$$

 En este caso, usar la distancia euclídea nos va a dar los mismos resultados que la distancia coseno.

Nótese que esta normalización es parecida al cálculo TF que compensa en la representación BoW la longitud de los documentos. 

Así que comencemos normalizando nuestros datos para que cada vector tenga norma unidad (vamos a hacerlo sobre las reprentaciones sparse).


In [ ]:
from scipy import sparse

def normalize_sparse_vector(s):
  norm_s = np.array(np.sqrt(s.multiply(s).sum(1)))
  pos_zero = np.where(np.sqrt(s.multiply(s).sum(1))==0)[0]
  norm_s[pos_zero] = 1
  return s.multiply(sparse.csr_matrix(1/norm_s))

corpus_bow_sparse_norm = normalize_sparse_vector(corpus_bow_sparse)
corpus_tfidf_sparse_norm = normalize_sparse_vector(corpus_tfidf_sparse)

Ahora claculemos el clustering con el K-means de sklearn

**Clustering de documentos con BoW**

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# K-means with K=4 (we could use the silhouette score to adjust this parameter) and 10 initializations
K=4
kmeans = KMeans(n_clusters=K, n_init=10) # Definimos objeto con parámetros por defecto

kmeans.fit(corpus_bow_sparse_norm) # Entrenamos k-means usando el BoW
y_kmeans = kmeans.predict(corpus_bow_sparse_norm) # Obtenemos el identificador del grupo para cada dato
centers = kmeans.cluster_centers_ # Obtenemos los centroides

In [ ]:
# Palabras más relevantes por centroide
n_center = 0
fileids = inaugural.fileids()
y_kmeans = np.array(y_kmeans)
for i, center_i in enumerate(centers):
  print('Centroide %d' %(i))
  # Find documents in this centroid
  pos_y = np.where(y_kmeans==i)[0]
  print('Elementos grupo %d' %(i))
  id_grupo = [fileids[pos] for pos in pos_y]
  print(id_grupo)

  # Find the ten most relevant words for this centroid
  pos_sort = np.argsort(centers[i])[::-1][:10]
  words_center = [D[p] for p in pos_sort]
  print(words_center)
  print('********')

**Clustering de documentos con TF-IDF**

In [ ]:
# K-means with K=4 (we could use the silhouette score to adjust this parameter) and 10 initializations
K=4
kmeans = KMeans(n_clusters=K, n_init=10) # Definimos objeto con parámetros por defecto

kmeans.fit(corpus_tfidf_sparse_norm) # Entrenamos k-means usando el BoW
y_kmeans = kmeans.predict(corpus_tfidf_sparse_norm) # Obtenemos el identificador del grupo para cada dato
centers = kmeans.cluster_centers_ # Obtenemos los centroides

In [ ]:
# Palabras más relevantes por centroide
n_center = 0
fileids = inaugural.fileids()
y_kmeans = np.array(y_kmeans)
for i, center_i in enumerate(centers):
  print('Centroide %d' %(i))
  # Find documents in this centroid
  pos_y = np.where(y_kmeans==i)[0]
  print('Elementos grupo %d' %(i))
  id_grupo = [fileids[pos] for pos in pos_y]
  print(id_grupo)

  # Find the ten most relevant words for this centroid
  pos_sort = np.argsort(centers[i])[::-1][:10]
  words_center = [D[p] for p in pos_sort]
  print(words_center)
  print('********')